# Clustering xerárquico

## Dependencias

In [1]:
!pip install numpy matplotlib scikit-learn seaborn pandas

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 11.2/11.2 MB 13.4 MB/s eta 0:00:00
   ---------------------------------------- 11.6/11.6 MB 16.5 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
   ---------------------------------------- 46.2/46.2 MB 16.9 MB/s eta 0:00:00


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering
import warnings

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
warnings.filterwarnings("ignore")

## Segmentación de clientes cun algoritmo de *clustering* xerárquico

Usaremos o algoritmo de *clustering* xerárquico aglomerativo para traballar co mesmo *dataset* de clientes do [notebook](02_Clustering.ipynb) da sesión anterior.

### Datos

#### Exploración dos datos

Descargamos o dataset co que traballaremos.

In [ ]:
!wget http://fegalaz.usc.es/~sdocio/apau2/p1/datasets/customers.csv

In [ ]:
df = pd.read_csv(r'customers.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.drop(columns=["CustomerID"]).describe()

In [ ]:
df.dtypes

Comprobamos que non hai valores nulos no *dataset*.

In [ ]:
df.isnull().sum()

### Segmentación usando perfil de gasto e ingresos anuais

Eliminamos as columnas coas que non imos traballar, como `CustomerID` (variable categórica que actúa como identificador único), `Gender` (variable categórica de tipo binario) e `Age`.

In [ ]:
df.drop(columns=["CustomerID", "Gender", "Age"], inplace=True)

In [ ]:
df.head()

Normalizamos os datos para axustar a escala.

In [ ]:
scaler = StandardScaler()

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
X = scaler.fit_transform(df[numeric_cols])

### Clustering

In [ ]:
method = 'average'
threshold = 0.7
distance = 'euclidean'
np.set_printoptions(suppress=True, precision=10)

Producimos a matriz de ligazóns (*linkage matrix*), definindo un método (`single`, `complete`, `average`, etc) e unha métrica (`euclidean`, `cosine`, etc). O resultado é unha matriz con forma `(n-1, 4)`, onde cada fila representa cada unha das fusións levadas a cabo nas sucesivas iteracións do proceso de agrupamento xerárquico. As catro columnas son:
- Columnas 0 e 1: índices dos *clusters* que se fusionan en cada iteración. Se os valores están entre `0` e `n-1`, refírese a un punto orixinal do dataset. Se é maior ou igual a `n`, refírese a un *cluster* novo, creado nun agrupamento anterior que pode combinar puntos individuais e/ou *clusters*.
- Columna 2: distancia entre os *clusters* fusionados.
- Columna 3: número total de puntos no novo *cluster* formado pola fusión. O valor vai aumentando a medida que se van fusionando os *clusters*. Na última fila o valor será `n`, xa que é un único *cluster* contendo todos os puntos de datos.

In [ ]:
Z = sch.linkage(X, method=method, metric=distance)

In [ ]:
Z[-1]


In [ ]:
trh = threshold * max(Z[:,2])

plt.figure(1, figsize=(15, 8))
plt.xlabel('Clientes')
plt.ylabel('Distancia')
plt.axhline(y=trh, color='r', linestyle='--', label=f'Limiar de corte ({trh:.2f})')
dendrogram = sch.dendrogram(Z, color_threshold=trh)

In [ ]:
agg = AgglomerativeClustering(n_clusters=None, distance_threshold=trh, metric='euclidean', linkage=method)
cluster_id = agg.fit(X)

In [ ]:
plt.figure(figsize=(15, 7))
plt.scatter(x='Annual Income (k$)', y='Spending Score (1-100)', data=df, c=agg.labels_, s=100, cmap='viridis', edgecolors='k')
plt.xlabel('Ingresos anuais (k$)')
plt.ylabel('Perfil de gasto (1-100)')
plt.show()